## Answering Business Questions using SQL
---

### Inroduction

In this guided project, we'll  use the Chinook database that is provided as a SQLite database file called `chinook.db`. The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

First of all we connect to database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Now let's check database tables.

In [2]:
%%sql
SELECT name, type
  FROM sqlite_master
 WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


There is database schema below.

![Image](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

In [3]:
%%html
<!-- left aligment for the table below -->
<style>
  table {margin-left: 0 !important;}
</style>

### Popular genres in the USA

Let's pretend tha the Chinook store has signed a deal with a record label. This label is able to add three albums from a list four:

| Artist Name          | Genre   |
|----------------------|---------|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

Our task is to pick most profitable albums. For tha we'll find what genre is the most popular one by checking number of tracks sold.
Since the label specializes in artists from **USA** we'll check only sales in this country.

In [4]:
%%sql
WITH invoice_tracks AS
                 (SELECT il.invoice_id, il.track_id,
                         il.quantity AS track_qty
                    FROM invoice_line AS il
                         INNER JOIN invoice AS i
                         ON il.invoice_id = i.invoice_id
                  
                   WHERE i.billing_country = "USA"
                 ),
    
     track_genres AS
                 (SELECT t.track_id,
                         g.name AS genre
                    FROM track AS t
                         INNER JOIN genre AS g
                         ON t.genre_id = g.genre_id
                 )

SELECT tg.genre,
       SUM(it.track_qty) AS tracks_sold,
       ROUND(CAST(SUM(it.track_qty) AS Float)*100/(
                                               SELECT SUM(it.track_qty)
                                                 FROM invoice_tracks AS it
                                                   ),
                      2) || "%" AS tracks_sold_pct
  FROM invoice_tracks AS it
       LEFT JOIN track_genres AS tg
       ON tg.track_id = it.track_id
        
 GROUP BY 1
 ORDER BY 2 DESC;

Done.


genre,tracks_sold,tracks_sold_pct
Rock,561,53.38%
Alternative & Punk,130,12.37%
Metal,124,11.8%
R&B/Soul,53,5.04%
Blues,36,3.43%
Alternative,35,3.33%
Latin,22,2.09%
Pop,22,2.09%
Hip Hop/Rap,20,1.9%
Jazz,14,1.33%


We've got a chart which shows us the amount of tracks sold by genre. Now we can use it to pick three albums from the label list simply comparing it with the chart.Here our recomendations:
* **Red Tone** - punk(and also alternative) tracks give **12.37%** of sales  
* **Slim Jim Bites** - blues tracks give **3.43%** of sales
* **Meteor and the Girls** - pop tracks give **2.09%** of sales

These three albums should bring more profit to the Chinook store. The remain album - **Regal** is hip-hop genre. This genre gives only **1.9%** of sales. 

### Best sales support agent

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We've been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

Let's write the corresponding query.

In [5]:
%%sql
WITH customer_sales AS
                 (SELECT c.support_rep_id, c.customer_id,
                         i.total
                    FROM customer AS c
                         LEFT JOIN invoice AS i
                         ON c.customer_id = i.customer_id
                 )

SELECT e.first_name || " " || e.last_name AS employee_name,
       DATE() - e.birthdate AS age,
       DATE(e.hire_date) AS hire_date,
       ROUND(SUM(cs.total), 0) AS sales,
       COUNT(DISTINCT cs.customer_id) AS num_of_customers,
       ROUND(AVG(cs.total), 2) AS avg_single_sale,
       ROUND(SUM(cs.total)/COUNT(DISTINCT cs.customer_id), 2) AS avg_sale_per_customer
  FROM employee AS e
       INNER JOIN customer_sales AS cs
       ON e.employee_id = cs.support_rep_id
        
 GROUP BY 1
 ORDER BY 4 DESC;

Done.


employee_name,age,hire_date,sales,num_of_customers,avg_single_sale,avg_sale_per_customer
Jane Peacock,48,2017-04-01,1732.0,21,8.17,82.45
Margaret Park,74,2017-05-03,1584.0,20,7.4,79.2
Steve Johnson,56,2017-10-17,1394.0,18,7.41,77.44


According the table above **Jane Peacock** is the best sales support agent. She has highest stats:
* number of customes - **21**
* average single sale - **8.17**
* average sales per customer - **82.45**

She's the most expirienced agent because she was hired first. And also she's the youngest one. Probably it explains her high results.

### Country sales analysys

Our next task is to analyze the sales data for customers from each different country using value from the **customers table**. In particular, you have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Also we should group all countries with only one customer as **"Other"** and put this group at the end of table.
Let's write a query.

In [6]:
%%sql
WITH other_countries AS
                 (SELECT AVG(i.total) AS avg_order_value,
                         SUM(i.total) AS total,
                         COUNT(DISTINCT c.customer_id) AS customers,
                         CASE
                             WHEN COUNT(DISTINCT c.customer_id) = 1 THEN "Other"
                             ELSE c.country
                         END AS country
                    FROM customer AS c
                         LEFT JOIN invoice AS i
                         ON c.customer_id = i.customer_id
                   GROUP BY c.country
                 ),
    
     world AS
            (SELECT AVG(i.total) AS avg_order_value,
                    SUM(i.total) AS total,
                    COUNT(DISTINCT c.customer_id) AS customers,
                    "World" AS country
               FROM customer AS c
                    LEFT JOIN invoice AS i
                    ON c.customer_id = i.customer_id
            ),
            
     final_countries AS
            (SELECT *
               FROM other_countries
            
             UNION
            
             SELECT *
               FROM world
            ) 
             

SELECT country,
       SUM(customers) AS customers,
       ROUND(SUM(total), 2) AS total_sales_value,
       ROUND(SUM(total)*100/(
                             SELECT SUM(total)
                               FROM other_countries
                            ),
                 2) || "%" AS total_sales_pct,
       ROUND(SUM(total)/SUM(customers), 2) AS avg_sales_per_customer,
       ROUND(AVG(avg_order_value), 2) AS avg_order_value
  FROM final_countries
 GROUP BY 1
 ORDER BY CASE
              WHEN country = "Other" THEN 1
              WHEN country = "World" THEN 2
              ELSE 0
          END, 3 DESC;

Done.


country,customers,total_sales_value,total_sales_pct,avg_sales_per_customer,avg_order_value
USA,13,1040.49,22.09%,80.04,7.94
Canada,8,535.59,11.37%,66.95,7.05
Brazil,5,427.68,9.08%,85.54,7.01
France,5,389.07,8.26%,77.81,7.78
Germany,4,334.62,7.11%,83.65,8.16
Czech Republic,2,273.24,5.8%,136.62,9.11
United Kingdom,3,245.52,5.21%,81.84,8.77
Portugal,2,185.13,3.93%,92.56,6.38
India,2,183.15,3.89%,91.57,8.72
Other,15,1094.94,23.25%,73.0,7.45


There are some quite interesting findings.

1. Chinook gained from sales in the USA about **1040** dollars - **22%** of all profit. It's comparable with profit gained from **15** different countries that bring **1095** dollars - **23%**
2. Czech Republic customers are real melomans. There are the highest average order value - **9.11** and average sales per customer - **136.62**
3. In addition we've got values for whole **world**. Now we can easily compare sales results from any country with global results.

### Individual tracks vs whole albums

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue

In [7]:
%%sql
WITH tracks_invoice AS
                  (SELECT il.invoice_id,
                          t.album_id,
                          il.track_id                          
                     FROM invoice_line AS il
                          LEFT JOIN track AS t
                          ON il.track_id = t.track_id
                  ),
    
     tracks_album AS
                  (SELECT a.album_id,
                          t.track_id
                     FROM album AS a
                          LEFT JOIN track AS t
                          ON a.album_id = t.album_id
                  )
            
SELECT CASE
           WHEN (
                 SELECT track_id
                   FROM tracks_invoice AS ti_in
                  WHERE ti_in.invoice_id = ti.invoice_id
                 EXCEPT
                 SELECT track_id
                   FROM tracks_album AS ta
                WHERE ta.album_id = ti.album_id
                 ) IS NULL
            AND (
                 SELECT track_id
                   FROM tracks_album AS ta
                 WHERE ta.album_id = ti.album_id
                 EXCEPT
                 SELECT track_id
                   FROM tracks_invoice AS ti_in
                 WHERE ti_in.invoice_id = ti.invoice_id
                ) IS NULL THEN "Full album"
            ELSE "Tracks"
        END AS invoice_type,
       COUNT(DISTINCT ti.invoice_id) AS invoice_qty,
       ROUND(COUNT(DISTINCT ti.invoice_id)*100/
                                              (SELECT CAST(COUNT(*) AS Float)
                                                 FROM invoice
                                              ), 1) || "%" AS invoice_pct
  FROM tracks_invoice AS ti
 GROUP BY 1;

Done.


invoice_type,invoice_qty,invoice_pct
Full album,114,18.6%
Tracks,500,81.4%


Only **18.6%** of sales are albumls. Other **81.4%** are generated by selling individual tracks.
According to that we can suggest that new purchasing stragegy will succeed and save Chinook's money.

### Most popular artist

To answer this question we'll use following criteria:
>  Artist should be used in the most playlists

Let's first create a view.

In [8]:
%%sql
CREATE VIEW track_artist AS
                                  SELECT ar.name, t.track_id, pt.playlist_id
                                     FROM track AS t
                                          LEFT JOIN album AS al
                                          ON t.album_id = al.album_id
                   
                                          LEFT JOIN artist AS ar
                                          ON al.artist_id = ar.artist_id
                    
                                          LEFT JOIN playlist_track AS pt
                                          ON t.track_id = pt.track_id
                                  

(sqlite3.OperationalError) table track_artist already exists
[SQL: CREATE VIEW track_artist AS
                                  SELECT ar.name, t.track_id, pt.playlist_id
                                     FROM track AS t
                                          LEFT JOIN album AS al
                                          ON t.album_id = al.album_id
                   
                                          LEFT JOIN artist AS ar
                                          ON al.artist_id = ar.artist_id
                    
                                          LEFT JOIN playlist_track AS pt
                                          ON t.track_id = pt.track_id]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


Now use our created view and criteria above.

In [9]:
%%sql
SELECT name,
       COUNT(DISTINCT playlist_id) AS in_playlist,
       ROUND(COUNT(DISTINCT playlist_id)*100/(
                                        SELECT CAST(COUNT(DISTINCT playlist_id) AS Float)
                                          FROM track_artist
                                       ), 1) || "%" AS in_playlist_pct
  FROM track_artist
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 10;

Done.


name,in_playlist,in_playlist_pct
Eugene Ormandy,7,50.0%
Academy of St. Martin in the Fields & Sir Neville Marriner,6,42.9%
Berliner Philharmoniker & Herbert Von Karajan,6,42.9%
English Concert & Trevor Pinnock,6,42.9%
The King's Singers,6,42.9%
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",5,35.7%
Adrian Leaper & Doreen de Feis,5,35.7%
Alberto Turco & Nova Schola Gregoriana,5,35.7%
Antal Doráti & London Symphony Orchestra,5,35.7%
Barry Wordsworth & BBC Concert Orchestra,5,35.7%


It seems **Eugene Ormandy**'s tracks appear in **7** playlists. But what if we would take different criterea:

> Artist's tracks should be used in the most playlists.

In [10]:
%%sql
SELECT name,
       COUNT(playlist_id) AS in_playlist,
       ROUND(COUNT(playlist_id)*100/(
                                    SELECT CAST(COUNT(playlist_id) AS Float)
                                      FROM track_artist
                                     ), 1) || "%" AS in_playlist_pct
  FROM track_artist
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 10;

Done.


name,in_playlist,in_playlist_pct
Iron Maiden,516,5.9%
U2,333,3.8%
Metallica,296,3.4%
Led Zeppelin,252,2.9%
Deep Purple,226,2.6%
Lost,184,2.1%
Pearl Jam,177,2.0%
Eric Clapton,145,1.7%
Faith No More,145,1.7%
Lenny Kravitz,143,1.6%


According the table above **"Iron Maden"** is the most popular artist. Their tracks appear **516** times in the playlists.

###  Purchased vs not purchased tracks

There are a lot of tracks in the Chinook store. Obviously some of then has never been sold. Let's find out how many.

In [11]:
%%sql
SELECT CASE
           WHEN il.invoice_id IS NULL THEN "Not purchased"
           ELSE "Purchased"
       END AS purchased_or_not,
       COUNT(DISTINCT t.track_id) AS track_qty,
       ROUND(COUNT(DISTINCT t.track_id)*100/(
                                    SELECT CAST(COUNT(*) AS Float)
                                      FROM track
                                   ), 1) || "%" AS track_pct
  FROM track AS t
       LEFT JOIN invoice_line AS il
       ON t.track_id = il.track_id
 GROUP BY 1;

Done.


purchased_or_not,track_qty,track_pct
Not purchased,1697,48.4%
Purchased,1806,51.6%


Now we can see that **48.4%** of tracks have never been sold. Let's explore genres of purchased tracks. We'll find how many tracks were sold from the stock in each genre.

In [99]:
%%sql
WITH tracks_genre_invoice AS
                      (SELECT g.name AS genre,
                              t.track_id,
                              il.invoice_id
                         FROM track AS t
                              LEFT JOIN genre AS g
                              ON t.genre_id = g.genre_id
                       
                              LEFT JOIN invoice_line AS il
                              ON t.track_id = il.track_id
                      ),
    
    sold_tracks_count AS
                      (SELECT tgi.genre,
                              COUNT(DISTINCT tgi.track_id) AS tracks_in_stock,
    
                              COUNT(DISTINCT CASE
                                                 WHEN tgi.invoice_id IS NOT NULL THEN tgi.track_id 
                                              END) AS tracks_sold,
        
                              ROUND(COUNT(DISTINCT CASE
                                                        WHEN tgi.invoice_id IS NOT NULL THEN tgi.track_id 
                                                     END
                                                        )*100/COUNT(DISTINCT tgi.track_id), 1) AS track_sold_pct
                         FROM tracks_genre_invoice AS tgi

                        GROUP BY 1
                      ),
    
    united_sales AS
                   (SELECT *
                      FROM sold_tracks_count

                    UNION

                    SELECT 'TOP_10' AS genre,
                           SUM(tracks_in_stock) AS tracks_in_stock,
                           SUM(tracks_sold) AS tracks_sold,
                           AVG(track_sold_pct) AS track_sold_pct
                      FROM (
                            SELECT *
                              FROM sold_tracks_count
                             ORDER BY 4 DESC
                             LIMIT 10
                           )
    
                    UNION

                    SELECT 'BOTTOM_10' AS genre,
                           SUM(tracks_in_stock) AS tracks_in_stock,
                           SUM(tracks_sold) AS tracks_sold,
                           AVG(track_sold_pct) AS track_sold_pct
                      FROM (
                            SELECT *
                              FROM sold_tracks_count
                             ORDER BY 4
                             LIMIT 10
                           )
                   )

SELECT *
  FROM united_sales
 ORDER BY CASE
              WHEN genre = "TOP_10" THEN 1
              WHEN genre = "BOTTOM_10" THEN 2
              ELSE 0
          END, 4 DESC;

Done.


genre,tracks_in_stock,tracks_sold,track_sold_pct
Easy Listening,24,24,100.0
Electronica/Dance,30,29,96.0
R&B/Soul,61,55,90.0
Alternative,40,34,85.0
Rock,1297,915,70.0
Blues,81,56,69.0
Metal,374,238,63.0
Hip Hop/Rap,35,21,60.0
Alternative & Punk,332,176,53.0
Pop,48,25,52.0


It's clearly that **10** genres are really unpopular. Only **11%** of tracks in the stock or less have ever been sold. I suggest to remove them from the chiinok store at all. It would be **312** tracks. Also it seems that some of them are not actual tracks at all.

Also I suggest to cut number of **Latin** tracks in half. It would be about **290** tracks.

Intsead of removed trakcks Chinnok store should **increase** number of track from **top 10** according to the table above.

### Protected vs non-protected media

The last question we are going to answer is

* Do protected vs non-protected media types have an effect on popularity?

We'll measure popularity by amount of sales.

In [107]:
%%sql
SELECT mt.name AS media_type,
       CASE
           WHEN il.invoice_id IS NULL THEN "Not purchased"
           ELSE "Purchased"
       END AS purchased_or_not,
       COUNT(DISTINCT t.track_id) AS track_qty,
       ROUND(COUNT(DISTINCT t.track_id)*100/(
                                    SELECT CAST(COUNT(*) AS Float)
                                      FROM track AS t
                                           LEFT JOIN media_type AS mt_in
                                           ON t.media_type_id = mt_in.media_type_id
                                     WHERE mt_in.name = mt.name
                                   ), 1) || "%" AS track_pct
  FROM track AS t
       LEFT JOIN invoice_line AS il
       ON t.track_id = il.track_id
        
       LEFT JOIN media_type AS mt
       ON t.media_type_id = mt.media_type_id
 GROUP BY 1, 2
HAVING mt.name LIKE "%Protected%";

Done.


media_type,purchased_or_not,track_qty,track_pct
Protected AAC audio file,Not purchased,86,36.3%
Protected AAC audio file,Purchased,151,63.7%
Protected MPEG-4 video file,Not purchased,211,98.6%
Protected MPEG-4 video file,Purchased,3,1.4%


There are only two types of protected files:

* Protected AAC audio file
* Protected MPEG-4 video file

Also let's remember that only **51.6%** of tracks were purchased from Chinook store.

**Protected MPEG-4 video** files are poorly sold. Only **1.4%** of tracks were sold. Actually these files are not tracks at all but video. Probably it should be removed from the store.

**Protected AAC audio** files have better results. **36.3%** of tracks were sold but it is still worse that average sales.

So protected files do not seem popular then unprotected ones. Even opposite.

### Conclusions

1. We've found most popular genres in the USA and made some recomendstions for the Chinook store:
    * **Red Tone** - punk(and also alternative) tracks give **12.37%** of sales  
    * **Slim Jim Bites** - blues tracks give **3.43%** of sales
    * **Meteor and the Girls** - pop tracks give **2.09%** of sales
    
2. We've found best sales agent. It's **Jane Peacock** with:
    * number of customes - **21**
    * average single sale - **8.17**
    * average sales per customer - **82.45**
    
3. We've analysed country sales and found following:
    * Chinook gained from sales in the USA about **1040** dollars - **22%** of all profit. It's comparable with profit gained from **15** different countries that bring **1095** dollars - **23%**
    * Czech Republic customers are real melomans. There are the highest average order value - **9.11** and average sales per customer - **136.62**
    * In addition we've got values for whole **world**. Now we can easily compare sales results from any country with global results.
    
4. We've explored album and individual tracks sales. Only **18.6%** of sales are albumls. Other **81.4%** are generated by selling individual tracks.

5. We've found most popular artists using two different criterias:
    * **Eugene Ormandy**. His tracks appear in **7** playlists
    * **"Iron Maden"**. Their tracks appear **516** times in the playlists
    
6. We've found that only **51.6%** of tracks have been sold. To improve this result we've suggested to cut number of **Latin** tracks in half and remove tracks of 10 genres from the store at all(less then **11%** were sold):
    - Soundtrack
    - TV Shows
    - Drama
    - Bossa Nova
    - Comedy
    - Opera
    - Rock And Roll
    - Sci Fi & Fantasy
    - Science Fiction
    - World
    
7. We've explored protected files from the Chinook store. They didn't bring more profit then unprotected ones. **Protected MPEG-4 video** files should be removed from the store at all. Only **1.4%** of them were sold